In [1]:
# Importing necessary packages
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import numpy as np
import datetime

In [2]:
# Instantiating Alpha Vantage variable
ts = TimeSeries(key = 'API_KEY_HERE')

In [3]:
# Creating an interval variable which will be used in the class
interval = '5min'

In [4]:
# Creating a class ScriptData that fetches stock data
class ScriptData:
    # Contructor for overloading
    def __init__(self, script):
        self.script = script
        self.fetch_intraday_data(self.script)
        self.df = self.convert_intraday_data(self.data)

    # Fetching raw stock data
    def fetch_intraday_data(self, script):
        self.data, meta_data = ts.get_intraday(script)
        return self.data

    # This method is used to fetch Intraday Historical day data
    # and convert it to a Data Frame
    def fetch_historical_data(self, script):
        data_01, meta_data_01 = ts.get_intraday(symbol=script, interval=interval, outputsize='full')
        self.data_list_01 = []
        for timestamp, items in data_01.items():
            open_item = float(items["1. open"])
            high_item = float(items["2. high"])
            low_item = float(items["3. low"])
            close_item = float(items["4. close"])
            volume = int(items["5. volume"])
            self.data_list_01.append([timestamp, open_item, high_item, low_item, close_item, volume])
        df_02 = pd.DataFrame(self.data_list_01, columns=["timestamp", "open", "high", "low", "close", "volume"])
        df_02["timestamp"] = pd.to_datetime(df_02["timestamp"])
        return df_02

    # Tis method converts the raw data that has been fetched
    # by fetch_intraday_data method and converts it into a Data Frame
    def convert_intraday_data(self, data):
        self.data_list = []
        for timestamp, items in data.items():
            open_item = float(items["1. open"])
            high_item = float(items["2. high"])
            low_item = float(items["3. low"])
            close_item = float(items["4. close"])
            volume = int(items["5. volume"])
            self.data_list.append([timestamp, open_item, high_item, low_item, close_item, volume])
        df = pd.DataFrame(self.data_list, columns=["timestamp", "open", "high", "low", "close", "volume"])
        df["timestamp"] = pd.to_datetime(df["timestamp"])
        return df

    # This method calculates moving averages in close/close_data
    # and saves it in indicator column
    def indicator1(self, timestamp, dataf, timeperiod = 5):
        indicator_data = []
        final_output = {"timestamp" : [], "indicator" : []}
        upcount = 0
        downcount = timeperiod - 1
        # Below part is where the moving average calculation takes place
        while downcount < len(dataf):
            cal_temp = 0
            # This for_loop loops from range upcount to downcount + 1
            # In this case it goes from 0 to 5+1 that means 0 - 5 and 
            # both upcount and downcount increases for each while loop
            for i in range(upcount, downcount + 1):
                cal_temp = float(dataf["close"][i]) + cal_temp
            final_temp = cal_temp/timeperiod
            # Appending the calculated data to a list
            indicator_data.append(final_temp)
            if downcount + 1 >= len(dataf):
                break
            upcount += 1
            downcount += 1
        # Appending timestamp column to a dictionary
        for i in range(0, len(dataf)):
            final_output["timestamp"].append(dataf["timestamp"][i])
        # Appending moving average(final_temp) column to a dictionary
        for i in range(0, len(dataf)):
            if i < timeperiod - 1:
                final_output["indicator"].append("NaN")
            else:
                final_output["indicator"].append(indicator_data[i-timeperiod + 1])
        # Finally converting the dictionary to a DataFrame with columns
        # "timestamp" and "indicator" 
        df_1 = pd.DataFrame(final_output)
        df_1.fillna(value = np.nan, inplace = True)
        return df_1
    
    # Overloading methods
    def __getitem__(self, index):
        return self.df.loc[index]

    def __setitem__(self, index, value):
        self.df.iloc[index] = value

    def __contains__(self, value):
        return self.script == value

In [5]:
# Creating an object for calls ScriptData and passing parameter
# as it needs to be initiated for __init(self, script)
script_data = ScriptData("GOOGL")

In [6]:
# Calling convert_intraday_data() method with
# raw data present in fetch_intraday_data()
script_data.convert_intraday_data(script_data.fetch_intraday_data("GOOGL"))

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,97.0000,97.1200,97.000,97.1000,23421
1,2023-02-15 19:45:00,96.9399,97.0000,96.900,96.9800,24896
2,2023-02-15 19:30:00,97.0000,97.0100,96.910,96.9400,12484
3,2023-02-15 19:15:00,96.9300,97.0100,96.800,97.0000,23284
4,2023-02-15 19:00:00,96.9800,97.0000,96.930,96.9400,17077
...,...,...,...,...,...,...
95,2023-02-14 12:15:00,92.6600,93.0399,92.530,93.0350,1306004
96,2023-02-14 12:00:00,92.3300,92.7300,92.270,92.6607,1437391
97,2023-02-14 11:45:00,92.7400,92.7400,92.260,92.3531,1869190
98,2023-02-14 11:30:00,93.1400,93.2500,92.695,92.7300,1506119


In [7]:
#Printing out the DataFrame
script_data.df

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,97.0000,97.1200,97.000,97.1000,23421
1,2023-02-15 19:45:00,96.9399,97.0000,96.900,96.9800,24896
2,2023-02-15 19:30:00,97.0000,97.0100,96.910,96.9400,12484
3,2023-02-15 19:15:00,96.9300,97.0100,96.800,97.0000,23284
4,2023-02-15 19:00:00,96.9800,97.0000,96.930,96.9400,17077
...,...,...,...,...,...,...
95,2023-02-14 12:15:00,92.6600,93.0399,92.530,93.0350,1306004
96,2023-02-14 12:00:00,92.3300,92.7300,92.270,92.6607,1437391
97,2023-02-14 11:45:00,92.7400,92.7400,92.260,92.3531,1869190
98,2023-02-14 11:30:00,93.1400,93.2500,92.695,92.7300,1506119


In [8]:
# Checking if "GOOGL" is in script_data
"GOOGL" in script_data

True

In [9]:
# Same as previous statements but passing "AAPL" symbol here
script_data = ScriptData("AAPL")
script_data.convert_intraday_data(script_data.fetch_intraday_data("AAPL"))

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,155.4399,155.510,155.4399,155.4800,17287
1,2023-02-15 19:45:00,155.4600,155.500,155.4000,155.4000,19333
2,2023-02-15 19:30:00,155.4800,155.490,155.4400,155.4700,9288
3,2023-02-15 19:15:00,155.3100,155.500,155.3000,155.4800,18221
4,2023-02-15 19:00:00,155.3900,155.390,155.3200,155.3300,7452
...,...,...,...,...,...,...
95,2023-02-14 12:15:00,151.6950,152.120,151.5700,151.9700,1337676
96,2023-02-14 12:00:00,151.1300,151.830,151.1000,151.7000,1509483
97,2023-02-14 11:45:00,151.5500,152.030,151.0400,151.1400,2000947
98,2023-02-14 11:30:00,152.0300,152.590,151.4400,151.5418,1951865


In [10]:
script_data.df

,timestamp,open,high,low,close,volume
0,2023-02-15 20:00:00,155.4399,155.510,155.4399,155.4800,17287
1,2023-02-15 19:45:00,155.4600,155.500,155.4000,155.4000,19333
2,2023-02-15 19:30:00,155.4800,155.490,155.4400,155.4700,9288
3,2023-02-15 19:15:00,155.3100,155.500,155.3000,155.4800,18221
4,2023-02-15 19:00:00,155.3900,155.390,155.3200,155.3300,7452
...,...,...,...,...,...,...
95,2023-02-14 12:15:00,151.6950,152.120,151.5700,151.9700,1337676
96,2023-02-14 12:00:00,151.1300,151.830,151.1000,151.7000,1509483
97,2023-02-14 11:45:00,151.5500,152.030,151.0400,151.1400,2000947
98,2023-02-14 11:30:00,152.0300,152.590,151.4400,151.5418,1951865


In [11]:
"AAPL" in script_data

True

In [12]:
# Creating a class Strategy which processes on Historical day data
class Strategy:
    # Constructor
    def __init__(self, script_data):
        self.script_data = script_data
    # Fetching historical data data frame from ScriptData class
    def fetch_intraday_historical_day_data(self, symbol):
        a = self.script_data.fetch_historical_data(symbol)
        close_values = [i for i in a["close"]]
        b = self.script_data.indicator1(a["timestamp"], a)
        # Changing the column names of 
        # "indicator" to "indicator_data"
        # "close" to "close_data", here we directly store data from close
        b = b.rename(columns={'indicator': 'indicator_data'})
        for i in range(0, len(a["close"])):
            b["close_data"] = close_values
        return b

    # In this method we calculate BUY, SELL and NO SIGNAL based on 
    # "indicator_data" and "close_data" and append it to a column called "signal"
    def get_signal(self, symbol):
        a = self.script_data.fetch_historical_data(symbol)
        close_values = [i for i in a["close"]]
        b = self.script_data.indicator1(a["timestamp"], a)
        b = b.rename(columns={'indicator': 'indicator_data'})
        for i in range(0, len(a["close"])):
            b["close_data"] = close_values
        c = {"timestamp" : [], "signal" : []}
        for i in range(0, len(b)):
            if float(b["indicator_data"].iloc[i]) < float(b["close_data"].iloc[i]):
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("SELL")
            elif float(b["indicator_data"].iloc[i]) > float(b["close_data"].iloc[i]):
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("BUY")
            else:
                c["timestamp"].append(b["timestamp"][i])
                c["signal"].append("NO SIGNAL")
        df_03 = pd.DataFrame(c)
        # Removing any row with "NO SIGNAL" and resetting the index
        df_03 = df_03[df_03['signal'] != 'NO SIGNAL'].reset_index(drop=True)
        # Uncomment this incase you want to get the output in excel file
        # df_03.to_excel('output.xlsx', index=True)
        return df_03

In [13]:
# Creating an object for Strategy and passing an argument/parameter
strategy_data = Strategy(script_data)

In [14]:
strategy_data.get_signal("GOOGL")

,timestamp,signal
0,2023-02-15 19:40:00,BUY
1,2023-02-15 19:35:00,BUY
2,2023-02-15 19:30:00,BUY
3,2023-02-15 19:25:00,BUY
4,2023-02-15 19:20:00,SELL
...,...,...
3839,2023-01-18 04:30:00,BUY
3840,2023-01-18 04:25:00,SELL
3841,2023-01-18 04:15:00,SELL
3842,2023-01-18 04:10:00,SELL
